## Source

https://github.com/ffreller/BirdShazam/blob/main/birds_download_and_prepare.ipynb

## Gather audio metadata

In [5]:
import pandas as pd
import requests
from tqdm import tqdm
import os

dataset_dir = '/home/colin/elec5305/ele5305_research_project/swiss_birds'
os.makedirs(dataset_dir,exist_ok=True)

country = 'switzerland'
response = requests.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}')
js = response.json()

df = pd.DataFrame()
for n_page in tqdm(range(1, js["numPages"]+1)):
    response = requests.get(f'https://www.xeno-canto.org/api/2/recordings?query=cnt:{country}&page={n_page}')
    page_json = response.json()
    ids, files, file_names, ens, lengths, gens = [], [], [], [], [], []
    for recording in page_json["recordings"]:
        ids.append(recording["id"])
        files.append(recording["file"])
        file_names.append(recording["file-name"])
        ens.append(recording["en"])
        lengths.append(recording["length"])
        gens.append(recording["gen"])


    df_ = pd.DataFrame.from_records({'id': ids, "file": files, "file-name": file_names, "en": ens, "gen": gens, "length": lengths})
    # df = df.append(df_)
    df = pd.concat([df, df_], ignore_index=True)


df.to_csv(os.path.join(dataset_dir,"birds.csv"), index=False)

100%|██████████| 7/7 [00:59<00:00,  8.45s/it]


## Download audio files

In [10]:
from pandas import read_csv
from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import os


df = read_csv(os.path.join(dataset_dir,"birds.csv"))

#Removing recordings that are not classified
df = df[df['en'] != 'Identity unknown'].copy()
df = df[df['en'] != 'Soundscape'].copy()

counts = df['en'].value_counts()
chosen = counts.head(25).index
df = df[df["en"].isin(chosen)]
df.to_csv(os.path.join(dataset_dir,"to_download.csv"), index=False)

print(df.shape)
print(len(df["en"].unique()), "different species")

audio_dir = os.path.join(dataset_dir,'audio_files/')
if not os_path.isdir(audio_dir):
    mkdir(audio_dir)

nots = []

for row in tqdm(df.iterrows(), total=df.shape[0]):
    url = row[1]["file"]
    f_name = str(row[1]["id"]) + row[1]["file-name"][-4:].lower()
    try:
        urlretrieve(url, filename=audio_dir + f_name)
    except Exception as e:
        print("\nRetrying:", url)
        print(e) 
        sleep(60)
        try:
            urlretrieve(url, filename=audio_dir + f_name)
        except Exception as ee:
            print("Not downloaded|", f_name)
            print(ee)
            nots.append(row[1]["id"])
            pass

if len(nots) > 0:
    print(len(nots))
    with open(os.path.join(dataset_dir,'not_downloaded.txt'), 'w') as f:
        for item in nots:
            f.write(str(item) + '\n')
    print(str(nots))
else:
    print('All files were successfully downloaded!')

(1155, 6)
25 different species



Retrying: https://xeno-canto.org/357351/download
HTTP Error 500: Internal Server Error


Not downloaded| 357351.mp3
HTTP Error 500: Internal Server Error

Retrying: https://xeno-canto.org/357350/download
HTTP Error 500: Internal Server Error


Not downloaded| 357350.mp3
HTTP Error 500: Internal Server Error


TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'str'

## Adapt dataframes for failed Downloads

In [13]:
%pip install pydub
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from os import listdir, mkdir, path as os_path, remove as os_remove
from tqdm.notebook import tqdm

#Check downloaded files
files = listdir(audio_dir)
df = pd.read_csv(os.path.join(dataset_dir,'to_download.csv'))
df['file-name'] = df['file-name'].str.lower()
df['filename'] = df['id'].astype(str) + df['file-name'].str[-4:]
downloaded = df[df['filename'].isin(files)][['filename', 'en', 'length', 'id', 'gen']]
downloaded.to_csv(os.path.join(dataset_dir,'downloaded.csv'), index=False)
print('Not downloaded:', len(df) - len(downloaded))

9790.63s - pydevd: Sending message related to process being replaced timed-out after 5 seconds

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Not downloaded: 2


## Convert mp3 to wav if necessairy

In [21]:
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from os import listdir, mkdir, path as os_path, remove as os_remove
from tqdm.notebook import tqdm
from pandas import read_csv
from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import os

def to_wav(filename, n_channels, sample_rate, input_dir, output_dir, delmp3 = False):
    input_type = filename[-4:].lower()
    if input_type == '.mp3':
        sound = AudioSegment.from_mp3(os.path.join(input_dir,filename))
        new_fn = fn[:-4] + '.wav'
    elif input_type == '.wav':
        sound = AudioSegment.from_wav(os.path.join(input_dir,filename))
        new_fn = filename
    sound = sound.set_channels(n_channels)
    sound.set_frame_rate(sample_rate)
    sound.export(f"{output_dir}/{new_fn}", format="wav")
    if delmp3:
        if new_fn in listdir(output_dir):
            os_remove(f"{input_dir}/{filename}")
        else:
            print(f'Problem: {input_dir}/{filename}')

wav_dir = os.path.join(dataset_dir,'wav_files/')
if not os_path.isdir(wav_dir):
    mkdir(wav_dir)
    mkdir(os.path.join(wav_dir,  'test/'))
    mkdir(os.path.join(wav_dir,  'train/'))
    mkdir(os.path.join(wav_dir,  'all_wav_files/'))

for fn in tqdm(downloaded['filename']):
    try:
        to_wav(fn, n_channels=1, sample_rate=16000, input_dir=audio_dir, output_dir=os.path.join(wav_dir,'all_wav_files'), delmp3=True)
    except Exception as e:
        print(fn)
        print(e)

815175.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/815175.wav'


805455.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/805455.wav'


771964.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/771964.wav'


760386.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/760386.wav'


760384.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/760384.wav'


...
































































































































































































































































































































































































































































































































































































































































































































































100%|██████████| 1037/1037 [10:01<00:00,  1.73it/s]








































































798272.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/798272.wav'


771574.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/771574.wav'


756305.wav
[Errno 2] No such file or directory: '/home/colin/elec5305/ele5305_research_project/swiss_birds/audio_files/756305.wav'










































100%|██████████| 116/116 [01:23<00:00,  1.39it/s]


## Check .csv metadata

In [16]:
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from os import listdir, mkdir, path as os_path, remove as os_remove
from tqdm.notebook import tqdm
from pandas import read_csv
from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import os

dataset_dir = '/home/colin/elec5305/ele5305_research_project/swiss_birds'

all_metadata = pd.read_csv(os.path.join(dataset_dir,'downloaded.csv'))

# Replace .mp3 with .wav
all_metadata['filename'] = all_metadata['filename'].str.replace('.mp3', '.wav')

all_wav_files = listdir(os.path.join(dataset_dir,'wav_files/all_wav_files/'))

# Check if all wav files in metadata file
missing = 0
for wav_file in all_wav_files:
    # if wav_file in all_metadata['filename']:
    if any(wav_file in filename for filename in all_metadata['filename']):
        pass
    else:
        print('{} is not in downloaded.csv'.format(wav_file))
        missing += 1
print('Total missing entries in metadata: {}'.format(missing))

# Check missing files for entries in metadata
missing = 0
for meatdata_entry in all_metadata['filename']:
    # if wav_file in all_metadata['filename']:
    if any(meatdata_entry in filename for filename in all_wav_files):
        pass
    else:
        print('{} is not in .wav directory'.format(wav_file))
        missing += 1
print('Total missing entries in .wav directory: {}'.format(missing))

# Convert 0:00 time format to float seconds
for i in range(len(all_metadata)):
    minutes, seconds = map(int, all_metadata['length'].iloc[i].split(':'))

    # Calculate the total time in seconds
    total_seconds = minutes * 60 + seconds

    # Convert to a float
    time_in_seconds = float(total_seconds)

    all_metadata['length'].iloc[i] = time_in_seconds

# Save new .csv as metadata
all_metadata.to_csv(os.path.join(dataset_dir, 'metadata.csv'), index=False)



Total missing entries in metadata: 0
Total missing entries in .wav directory: 0


/tmp/ipykernel_34495/501600916.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_metadata['length'].iloc[i] = time_in_seconds
/tmp/ipykernel_34495/501600916.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_metadata['length'].iloc[i] = time_in_seconds
/tmp/ipykernel_34495/501600916.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_metadata['length'].iloc[i] = time_in_seconds
/tmp/ipykernel_34495/501600916.py:56: Se

## Perform Train / Test Split

In [20]:
from sklearn.model_selection import train_test_split
from pydub import AudioSegment
from os import listdir, mkdir, path as os_path, remove as os_remove
from tqdm.notebook import tqdm
from pandas import read_csv
from urllib.request import urlretrieve
from os import listdir, mkdir, path as os_path
from tqdm.notebook import tqdm
from time import sleep
import pandas as pd
import requests
from tqdm import tqdm
import os
import shutil

def move_wav_file(source_dir,target_dir,filenames: list):
    for filename in filenames:
        source_path = os.path.join(source_dir, filename)
        target_path = os.path.join(target_dir, filename)
        shutil.move(source_path, target_path)

    return

test_size = 0.1
train, test = train_test_split(all_metadata, stratify=all_metadata["en"], test_size=test_size)

# Save new .csv as train as test metadata
train.to_csv(os.path.join(dataset_dir, 'metadata_train.csv'), index=False)
test.to_csv(os.path.join(dataset_dir, 'metadata_test.csv'), index=False)

# Define Test and Train directories
test_dir = os.path.join(dataset_dir, 'wav_files/test/')
train_dir = os.path.join(dataset_dir, 'wav_files/train/')

# Make directories
os.makedirs(test_dir, exist_ok=True)
os.makedirs(train_dir, exist_ok=True)

# Copy .wav files to test folder
move_wav_file(source_dir=os.path.join(dataset_dir,'wav_files/all_wav_files/'), target_dir=test_dir, filenames = test['filename'])

# Copy .wav files to train folder
move_wav_file(source_dir=os.path.join(dataset_dir,'wav_files/all_wav_files/'), target_dir=train_dir, filenames = train['filename'])